## 4.3. Análise de tendências e sazonalidade da qualidade do ar no Brasil

        Tabela com estatísticas da população exposta para cada padrão de qualidade do ar por estado

        Tabela iterativa por estação de monitoramento e cidade

        Figura com estação de monitoramento que viola cada um dos padrões. Demonstrar buffer e população atingida/setores sensitários.